In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("./../..")

In [3]:
import numpy as np
import torch

#local
from misc import metrics as m
from effcn.layers import FCCaps

#### simulate capsule layer U

In [4]:
torch.manual_seed(0)
U = torch.rand(1,2,4)
print(U.shape)
print(U.max())
print(U.min())
print(U.mean())
print(U.std())

torch.Size([1, 2, 4])
tensor(0.8964)
tensor(0.0885)
tensor(0.4766)
tensor(0.2890)


In [5]:
Ur = torch.clone(U)
Ur[:,1,:] = 1
#Ur[:,1,:] = 0
print(Ur)

tensor([[[0.4963, 0.7682, 0.0885, 0.1320],
         [1.0000, 1.0000, 1.0000, 1.0000]]])


#### rate_dead_capsules_norm

In [6]:
m.rate_dead_capsules_norm(Ur)

array([False, False])

In [7]:
U_norm = np.linalg.norm(U, axis=2)
#U.mean(axis=0).shape

U_norm

array([[0.9282756, 1.2411162]], dtype=float32)

#### simulate coupling for layers

In [19]:
FCL = FCCaps(U.shape[1],4, U.shape[2], 5)

In [32]:
U_hat, A, A_scaled, A_sum, C, CB, U_h_fin, U_h_sq = FCL.forward_debug(Ur)

C.shape

torch.Size([1, 2, 4])

In [34]:
Cn = torch.clone(C).detach().numpy()
Cn[:,0,2] += 0.2
Cn[:,0,0] -= 0.2
print(Cn)

[[[0.03751105 0.23688202 0.4931419  0.23246495]
  [0.2767547  0.22670116 0.23900557 0.25753862]]]


In [35]:
n_samples, n_l, n_h = Cn.shape
ef = 1 / n_h + 1 / n_h * 0.5

print(n_h)
print(ef)

4
0.375


#### rate_inactive_capsules

In [39]:
pr = m.rate_inactive_capsules(Cn)

pr

array([0., 1.])

In [40]:
m.normalize_couplings(Cn)

(array([[[0.       , 0.       , 0.4931419, 0.       ],
         [0.       , 0.       , 0.       , 0.       ]]], dtype=float32),
 array([0., 1.]))

#### mm_couplings

In [43]:
m.mm_couplings(Cn)

array([0.4931419, 0.2767547], dtype=float32)

mma_couplings

In [50]:
mma = m.mma_couplings(Cn, pr)

print(mma)

[4.93141889e-01 2.76754707e+08]


In [53]:
print(1/(1 - pr.flatten() + 1e-9))
print(Cn.max(axis=-1).mean(axis=0))
print(Cn.max(axis=-1).mean(axis=0) / (1 - pr.flatten() + 1e-9))

[9.99999999e-01 1.00000000e+09]
[0.4931419 0.2767547]
[4.93141889e-01 2.76754707e+08]


In [49]:
m.lmma_couplings(Cn, pr)

0.4931418880927178

In [67]:
ws = (1 - pr) / (1 - pr + 1e-9).sum()
print((ws * mma).sum())
print((ws * mma))

print(ws)

0.4931418880927178
[0.49314189 0.        ]
[1. 0.]


In [57]:
m.ma_couplings(Cn, pr)

array([[3.75110507e-02, 2.36882016e-01, 4.93141889e-01, 2.32464954e-01],
       [2.76754707e+08, 2.26701155e+08, 2.39005566e+08, 2.57538617e+08]])

In [65]:
_, n_l, _ = Cn.shape
prx = pr.reshape(n_l, 1)
Cn.mean(axis=0) / (1 - prx + 1e-9)

array([[3.75110507e-02, 2.36882016e-01, 4.93141889e-01, 2.32464954e-01],
       [2.76754707e+08, 2.26701155e+08, 2.39005566e+08, 2.57538617e+08]])

In [68]:
m.stda_couplings(Cn, pr)

array([[3.75110567e-11, 2.36882042e-10, 4.93141916e-10, 2.32464964e-10],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [72]:
_, n_l, _ = Cn.shape
prx = pr.reshape(n_l, 1)
ma = m.ma_couplings(Cn, prx)
p1 = ((Cn - ma)**2).mean(axis=0) / (1 - prx + 1e-9)
p2 = ma**2 * prx / (1 - prx + 1e-9)
p3 = p1 - p2
p3 = np.maximum(0, p3)

print(ma)
print(p1)
print(p2)
print(p3)

[[3.75110507e-02 2.36882016e-01 4.93141889e-01 2.32464954e-01]
 [2.76754707e+08 2.26701155e+08 2.39005566e+08 2.57538617e+08]]
[[1.40707938e-21 5.61131017e-20 2.43188949e-19 5.40399594e-20]
 [7.65931677e+25 5.13934137e+25 5.71236603e+25 6.63261389e+25]]
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.65931679e+25 5.13934138e+25 5.71236604e+25 6.63261391e+25]]
[[1.40707938e-21 5.61131017e-20 2.43188949e-19 5.40399594e-20]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
